In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv1D, Dense, Flatten, MaxPooling1D, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight

# Load the CSV data
data = pd.read_csv(r'C:\Users\deven\Desktop\IMES\Project\Merged_df_v4.csv')

In [2]:
# Define input and output variables (excluding Frequency and Frequency_Location)
X = data[['Time', 'AccX_Filtered', 'AccY_Filtered', 'AccZ_Filtered', 'Section']]
y = data[['Damping_Location', 'Inclination_Location']]

In [3]:
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

X_scaled = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

In [5]:
# Define a helper function to create a model for Damping and Inclination Location
def create_location_model(output_units, output_activation, loss_function):
    inputs = Input(shape=(X_train.shape[1], 1))
    x = Conv1D(filters=32, kernel_size=2, activation='relu', padding='same')(inputs)
    x = MaxPooling1D(pool_size=2, padding='same')(x)
    x = Conv1D(filters=64, kernel_size=2, activation='relu', padding='same')(x)
    x = MaxPooling1D(pool_size=2, padding='same')(x)
    x = Conv1D(filters=128, kernel_size=2, activation='relu', padding='same')(x)
    x = MaxPooling1D(pool_size=2, padding='same')(x)
    x = Flatten()(x)
    outputs = Dense(output_units, activation=output_activation)(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Model for damping location
model_damp_loc = create_location_model(y_train['Damping_Location'].nunique(), 'softmax', 'binary_crossentropy')
model_damp_loc.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model_damp_loc.fit(X_train, to_categorical(y_train['Damping_Location']), epochs=10, batch_size=64, validation_split=0.2)

# Model for inclination location
model_incl_loc = create_location_model(y_train['Inclination_Location'].nunique(), 'softmax', 'binary_crossentropy')
model_incl_loc.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model_incl_loc.fit(X_train, to_categorical(y_train['Inclination_Location']), epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
4331/4331 [==============================] - 29s 6ms/step - loss: 0.1428 - accuracy: 0.9465 - val_loss: 0.1383 - val_accuracy: 0.9459
Epoch 2/10
4331/4331 [==============================] - 27s 6ms/step - loss: 0.1358 - accuracy: 0.9466 - val_loss: 0.1367 - val_accuracy: 0.9459
Epoch 3/10
4331/4331 [==============================] - 27s 6ms/step - loss: 0.1350 - accuracy: 0.9467 - val_loss: 0.1358 - val_accuracy: 0.9459
Epoch 4/10
4331/4331 [==============================] - 26s 6ms/step - loss: 0.1342 - accuracy: 0.9468 - val_loss: 0.1363 - val_accuracy: 0.9460
Epoch 5/10
4331/4331 [==============================] - 25s 6ms/step - loss: 0.1337 - accuracy: 0.9468 - val_loss: 0.1359 - val_accuracy: 0.9459
Epoch 6/10
4331/4331 [==============================] - 24s 6ms/step - loss: 0.1332 - accuracy: 0.9469 - val_loss: 0.1351 - val_accuracy: 0.9459
Epoch 7/10
4331/4331 [==============================] - 25s 6ms/step - loss: 0.1329 - accuracy: 0.9470 - val_loss: 0.1346 - val_ac

In [6]:
# Make predictions with each model
y_pred_damp_loc = model_damp_loc.predict(X_test)
y_pred_incl_loc = model_incl_loc.predict(X_test)

# Decode predictions
y_pred_damp_loc_decoded = np.argmax(y_pred_damp_loc, axis=1)
y_pred_incl_loc_decoded = np.argmax(y_pred_incl_loc, axis=1)

# Create a DataFrame to see the results
predictions_df = pd.DataFrame({
    'Predicted_Damping_Location': y_pred_damp_loc_decoded,
    'Predicted_Inclination_Location': y_pred_incl_loc_decoded
})

# Display the predictions
print(predictions_df.tail(10))

4641/4641 [==============================] - 11s 2ms/step
        Predicted_Damping_Location  Predicted_Inclination_Location
148480                           0                               0
148481                           0                               0
148482                           0                               0
148483                           0                               0
148484                           0                               0
148485                           0                               0
148486                           0                               0
148487                           0                               0
148488                           0                               0
148489                           0                               0


In [7]:
# predictions_df.to_csv(r'E:\RWTH\Sem2\IMES\Trainedset_v2.csv')

In [14]:
data = pd.read_csv(r'C:\Users\deven\Desktop\IMES\Project\Test_v1\Test_v2.csv')
data.shape

(49983, 10)

In [15]:
data = data.drop(['Group number', 'Case number', 'Absolute acceleration (m/s^2)','Frequency','Frequency_Location'], axis =1)
data.shape

(49983, 5)

In [16]:
data = data[~data.Section.isin([7,1])]
data.tail(5000)

,Time (s),Linear Acceleration x (m/s^2),Linear Acceleration y (m/s^2),Linear Acceleration z (m/s^2),Section
44983,24.543369,-2.737154,2.317677,28.762552,5
44984,24.548369,0.478224,0.850334,-10.163772,5
44985,24.553369,0.835628,-0.861210,-8.116422,5
44986,24.558369,0.940185,-1.352423,3.283479,5
44987,24.563369,0.216398,1.000855,-6.343028,5
...,...,...,...,...,...
49978,49.518370,-0.097531,0.246785,-0.046586,2
49979,49.523370,0.000420,0.305819,-0.467093,2
49980,49.528370,0.080823,-0.409701,0.436840,2
49981,49.533370,-0.072530,0.159599,-0.059052,2


In [17]:
imputer = SimpleImputer(strategy='mean')
data_imputed = imputer.fit_transform(data)

scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_imputed)

data_scaled = data_scaled.reshape((data_scaled.shape[0], data_scaled.shape[1], 1))

In [18]:
data_scaled.shape

(43999, 5, 1)

In [19]:
# Make predictions with each model
y_pred_damp_loc = model_damp_loc.predict(data)
y_pred_incl_loc = model_incl_loc.predict(data)

# Decode predictions
y_pred_damp_loc_decoded = np.argmax(y_pred_damp_loc, axis=1)
y_pred_incl_loc_decoded = np.argmax(y_pred_incl_loc, axis=1)

# Create a DataFrame to see the results
predictions_df = pd.DataFrame({
    'Predicted_Damping_Location': y_pred_damp_loc_decoded,
    'Predicted_Inclination_Location': y_pred_incl_loc_decoded
})

1375/1375 [==============================] - 4s 3ms/step


In [23]:
predictions_df['Predicted_Damping_Location'].unique()

array([0], dtype=int64)